In [1]:
import xpress as xp
import numpy as np
import pandas as pd
import time

In [2]:
print("Loading Data")
# Load distance matrix
distance_matrix = np.loadtxt('distance_matrix_reduce.csv', delimiter=',')
stations = pd.read_csv('candidate_stations_P300_kmeans_no_snap.csv')
pois = pd.read_csv('reduced_pois.csv')

Loading Data


In [3]:
I = distance_matrix.shape[0]  # Number of POIs (demand points)
J = distance_matrix.shape[1] # Number of candidate stations

In [6]:
print("\n" + "="*70)
print("SOLVING MODEL...")
print("="*70 + "\n")

results_list = []
p_values = range(1, J + 1)

for p in p_values:
    print(f"Solving for p = {p}...")
    
    #  CREATE NEW PROBLEM FOR EACH P
    xp.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')
    prob = xp.problem(name=f'p_centre_p{p}')
    
    # Recreate variables
    Y = {j: prob.addVariable(vartype=xp.binary, name=f'Y_{j}') for j in range(J)}
    X = {(i, j): prob.addVariable(vartype=xp.binary, name=f'X_{i}_{j}') 
         for i in range(I) for j in range(J)}
    W = prob.addVariable(name='W', lb=0)
    
    # Add ALL constraints
    for i in range(I):
        prob.addConstraint(xp.Sum(X[i, j] for j in range(J)) == 1)
    
    # Facility count constraint (FRESH, not accumulated)
    prob.addConstraint(xp.Sum(Y[j] for j in range(J)) == p)
    
    for i in range(I):
        for j in range(J):
            prob.addConstraint(X[i, j] <= Y[j])
    
    for i in range(I):
        prob.addConstraint(xp.Sum(distance_matrix[i, j] * X[i, j] for j in range(J)) <= W)
    
    # Set objective
    prob.setObjective(W, sense=xp.minimize)
    
    
    start_solve = time.time()
    
    # Use prob.optimize() for MIP
    xp.setOutputEnabled(False)
    prob.solve()
    
    solve_time = time.time() - start_solve
    
    # Extract W 
    W_optimal = prob.getSolution(W)

    stations_open = []
    for j in range(J):
        if prob.getSolution(Y[j]) > 0.5:  # Binary variable, so > 0.5 means 1
            stations_open.append(j)

    print(f"\n   Station details:")
    optimal_stations_list = []
    for idx, j in enumerate(stations_open, 1):
        station_id = stations.iloc[j]['candidate_id']
        lat = station_lat = stations.iloc[j]['centroid_lat']        
        lon = station_lon = stations.iloc[j]['centroid_lon']
        
        optimal_stations_list.append({
            'index': j,
            'rank': idx,
            'station_id': station_id,
            'latitude': lat,
            'longitude': lon
        })
        
        print(f"   {idx:2d}. (Station ID = {station_id})")

    # Append to results_list
    results_list.append(W_optimal)
    
    print(f"  W* = {W_optimal:.4f} km | Time = {solve_time:.1f}s")

print(f"\n✓ Results collected: {len(results_list)} instances")



SOLVING MODEL...

Solving for p = 1...

   Station details:
    1. (Station ID = 207)
  W* = 10.9736 km | Time = 5536.1s
Solving for p = 2...


SolverError: ?1081 Error: The Xpress-Optimizer license has been lost; this may be because of a connection failure with the license server.